In [1]:
import pandas as pd
import json
from google.cloud import bigquery

In [15]:
food_categories = [
'Restaurant',
'Fast food restaurant',
'Takeout Restaurant',
'Pizza restaurant',
'Mexican restaurant',
'Juice shop',
'Hamburger restaurant',
'Asian restaurant',
'Barbecue restaurant',
'Delivery Restaurant',
'Pizza Takeout',
'Seafood restaurant',
'Chicken restaurant',
'Lounge',
'Cake shop',
'Sushi restaurant',
'Salad shop',
'Taco restaurant',
'Bagel shop',
'Diner',
'Espresso bar',
'Latin American restaurant',
'Buffet restaurant',
'Tea house',
'Ice cream shop',
'Bakery',
'Coffee store',
'Wine bar',
'Caterer',
'Chicken wings restaurant',
'Candy store',
'Frozen yogurt shop',
'Wine store',
'Pastry shop',
'Food court',
'Fried chicken takeaway',
'Jamaican restaurant',
'New American restaurant',
'Greek restaurant',
'French restaurant',
'Salvadoran restaurant',
'Hawaiian restaurant',
'Organic restaurant',
'Soup restaurant',
'Grill',
'European restaurant',
'Chinese takeaway',
'Delivery Chinese restaurant',
'Dominican restaurant',
'Hot dog stand',
'Brewpub',
'Hot dog restaurant',
'Beer garden',
'Filipino restaurant',
'Pie shop',
'Colombian restaurant',
'Pakistani restaurant',
'Japanese steakhouse',
'Ramen restaurant',
'Sushi takeaway',
'Irish pub',
'Vietnamese restaurant',
'Ethiopian restaurant',
'Turkish restaurant',
'Creole restaurant',
'Korean barbecue restaurant',
'Cider bar',
'Oyster bar restaurant',
'Taiwanese restaurant',
'Hot pot restaurant',
'West African restaurant',
'Rice restaurant',
'Dim sum restaurant',
'Fusion restaurant',
'Mexican torta restaurant',
'Californian restaurant',
'Guatemalan restaurant',
'Sichuan restaurant',
'Teppanyaki restaurant',
'South American restaurant',
'Afghani restaurant',
'Ecuadorian restaurant',
'Pan-Asian restaurant',
'Peruvian restaurant',
'Authentic Japanese restaurant',
'Moroccan restaurant',
'East African restaurant',
'Russian restaurant',
'Laotian restaurant',
'Irish restaurant',
'Polish restaurant',
'Modern Indian restaurant',
'Cambodian restaurant',
'Mongolian barbecue restaurant',
'Izakaya restaurant',
'Japanese curry restaurant',
'Hunan restaurant',
'Neapolitan restaurant',
'Shanghainese restaurant',
'Fondue restaurant',
'Shawarma restaurant',
'Conveyor belt sushi restaurant',
'Pacific Northwest restaurant (US)',
'Pan-Latin restaurant',
'Poâ€™ boys restaurant',
'Egyptian restaurant',
'Israeli restaurant',
'Nicaraguan restaurant',
'Syrian restaurant',
'Polynesian restaurant',
'Scandinavian restaurant',
'French steakhouse restaurant',
'Sicilian restaurant',
'South African restaurant',
'Chilean restaurant',
'Brazilian pastelaria',
'Yemenite restaurant',
'Sri Lankan restaurant',
'Pacific Rim restaurant',
'Yucatan restaurant',
'Swiss restaurant',
'Austrian restaurant',
'Japanese regional restaurant',
'Costa Rican restaurant',
'Katsudon restaurant',
'Uruguayan restaurant',
'North African restaurant',
'Czech restaurant',
'Porridge restaurant',
'Korean rib restaurant',
'Obanzai restaurant',
'Haute French restaurant',
'Tempura restaurant',
'Tempura donburi restaurant',
'Sfiha restaurant',
'Sukiyaki restaurant',
'Valencian restaurant',
'Icelandic restaurant',
'Paraguayan restaurant',
'Provence restaurant',
'Scottish restaurant',
'Kushiyaki restaurant',
'Pueblan restaurant',
'Roman restaurant',
'Croatian restaurant',
'Kaiseki restaurant',
'Anago restaurant',
'Madrilian restaurant',
'Anhui restaurant',
'Shandong restaurant',
'Turkmen restaurant',
'Asian',
'Bars',
'Cocktail Bars',
'Restaurants',
'Wine Bars',
'Food',
'Vietnamese',
'Salad',
'Thai',
'Laotian',
'Noodles',
'Buffets',
'Chinese',
'Asian Fusion',
'Coffee & Tea',
'Pizza',
'Sandwiches',
'Italian',
'Cafes',
'Breakfast & Brunch',
'Steakhouses',
'Seafood',
'Mexican',
'Tacos',
'Sushi Bars',
'Vegetarian',
'Vegan',
'Southern',
'Cajun/Creole',
'Cucina campana',
'Pasta Shops',
'Burgers',
'Chicken Wings',
'Barbeque',
'Caribbean',
'Beer',
'Grocery',
'Ethnic Food',
'International Grocery',
'Malaysian',
'Cambodian',
'Greek',
'Bakeries',
'Hot Dogs',
'Fast Food',
'American (Traditional)',
'Juice Bars & Smoothies',
'Cupcakes',
'Patisserie/Cake Shop',
'Comfort Food',
'Desserts',
'Bagels',
'Gastropubs',
'Pubs',
'Japanese',
'Indian',
'French',
'Wraps',
'Acai Bowls',
'Latin American',
'Ice Cream & Frozen Yogurt',
'Donuts',
'Argentine',
'Basque',
'Tapas/Small Plates',
'Hawaiian',
'Poke',
'Food Stands',
'Farmers Market',
'Food Trucks',
'Gluten-Free',
'Kombucha',
'Diners',
'Korean',
'Soul Food',
'Sports Bars',
'German',
'Breweries',
'Ramen',
'Polish',
'Mediterranean',
'Turkish',
'Middle Eastern',
'Specialty Food',
'Beer Bar',
'Food Delivery Services',
'Fish & Chips',
'Cuban',
'Cheesesteaks',
'Falafel',
'Halal',
'Kebab',
'Bubble Tea',
'Waffles',
'Soup',
'Tiki Bars',
'Salvadoran',
'Macarons',
'Mongolian',
'Pakistani',
'Gelato',
'Wineries',
'Modern European',
'Pretzels',
'Karaoke',
'Dim Sum',
'Hot Pot',
'Supper Clubs',
'Dance Clubs',
'Themed Cafes',
'Szechuan',
'Armenian',
'Cannabis Clinics',
'Mobile Phones',
'Convenience Stores',
'Automotive',
'Delis',
'Brewpubs',
'Live/Raw Food',
'Tex-Mex',
'African',
'Dive Bars',
'Irish',
'Fruits & Veggies',
'Meat Shops',
'Pan Asian',
'Israeli',
'Waffles']


In [2]:
dataset_id = 'tu dataset'
table_id = 'tu tabla'

In [4]:
client = bigquery.Client()
state_query = f"""
SELECT DISTINCT us_state AS state
FROM `{{}}.{{}}`
""".format(dataset_id, table_id)

In [16]:
state_query_job = client.query(state_query)
unique_states = [row.state for row in state_query_job]
state_category_counts = {}


for state in unique_states:
    state_query = f"""
    SELECT
      us_state AS state,
      categories AS restaurant_categories
    FROM `{dataset_id}.{table_id}`
    WHERE us_state = '{state}'
    """

    state_query_job = client.query(state_query)
    state_df = state_query_job.to_dataframe()

    category_count = {}

    for index, row in state_df.iterrows():
        categories = row['restaurant_categories']

        if isinstance(categories, str):
            if categories in food_categories:
                if categories in category_count:
                    category_count[categories] += 1
                else:
                    category_count[categories] = 1
        elif isinstance(categories, dict):
            data_list = categories['list'].tolist()
            for category_obj in data_list:
                value = category_obj['element']

                if value in food_categories:
                    if value in category_count:
                        category_count[value] += 1
                    else:
                        category_count[value] = 1

    state_category_counts[state] = category_count

with open('state_category_counts.json', 'w') as json_file:
    json.dump(state_category_counts, json_file, indent=4)

State: WA
Grill: 811
Beer garden: 408
Restaurant: 50304
Hamburger restaurant: 4050
Lounge: 1569
Brewpub: 1263
Bakery: 4612
Organic restaurant: 387
Espresso bar: 5910
Fast food restaurant: 19172
Caterer: 14850
Takeout Restaurant: 17243
Vietnamese restaurant: 1545
Diner: 1304
Cake shop: 1693
Pastry shop: 601
Tea house: 1421
Filipino restaurant: 232
Russian restaurant: 85
Wine bar: 1797
Cider bar: 505
Pizza restaurant: 12214
Karaoke: 96
Bagel shop: 386
Wine store: 407
Juice shop: 2551
Mexican restaurant: 15372
Taco restaurant: 1061
Ice cream shop: 2363
Soup restaurant: 1135
Salad shop: 1773
Coffee store: 4051
Delivery Restaurant: 4955
Frozen yogurt shop: 696
Asian restaurant: 6752
Pizza Takeout: 3816
Chicken wings restaurant: 3460
Sushi restaurant: 2838
Greek restaurant: 744
Chicken restaurant: 3962
European restaurant: 290
Izakaya restaurant: 194
Seafood restaurant: 1221
Buffet restaurant: 2358
Dim sum restaurant: 224
Barbecue restaurant: 432
Hot pot restaurant: 124
New American restaura